In [ ]:
import pandas as pd
import scipy as sp
from scipy.sparse import diags
import numpy as np
from numpy import linalg as LA
import sys
from os import path

import matplotlib.pyplot as plt

#importing seaborn for plotting
import seaborn as sns

#for plotting purposes
%pylab inline
sns.set_style('ticks')
sns.set_context('paper')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib as mpl

# mpl.rcParams
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['figure.figsize'] = [8, 16/3]

## functions for use in processing

In [ ]:
#porting from HITRACE
#based on Baseline correction for NMR spectroscopic metabolomics data analysis. 2008, 
#Author(s): Xi, Yuanxin and Rocke, David M
#https://doi.org/10.1186/1471-2105-9-324 and further modified by Rhiju (Stanford University)
#name of variables tracks closely to what is presented in the manuscript

def baseline_xi(b,A=2e9,B=2e4,s=1.0):
    # Input: 
    # b Nx1, spectrum data
    # A 1x1 smoothing factor
    # B 1x1 negative penalty
    # s 1x1 noise standard deviation

    # Output:
    # bd Nx1 baseline
    # b_subtract Nx1 background subtracted trace

    L = len(b)

    # b is the SIGNAL (gamma in the paper). bd is the estimated baseline (b in the paper).
    bd = np.ones((L,1))*np.median(b)
    bd0 = b

    #current error
    nm = LA.norm(b-bd0)
    nm0 = sys.float_info.max #initialize with largest possible float

    #solving D*bd = m
    #D and m have been divided through by A
    #Mistake in expression for M; should be +1, not -1
    M0 = s*np.ones((L,1))/A

    #initialize D matrix
    e = np.ones((L,1))
    diagonals = [2, -8, 12, -8, 2]
    D0 = diags(diagonals, np.arange(-2,3), shape=(L,L)).toarray()
    
    D0[0,0] = 2
    D0[L-1,L-1] = 2

    D0[1,0] = -4
    D0[0,1] = -4
    D0[L-1,L-2] = -4
    D0[L-2,L-1]= -4

    D0[1,1] = 10
    D0[L-2,L-2] = 10

    #index for iteration
    i=0

    while ((nm>10 or i<5) and i<30):
        i=i+1
        M = M0
        D = D0
        bd0 = bd
        nm0=nm #not sure this is needed, nm0 not used in iteration

        for j in np.arange(0,L):
            if (bd[j]>b[j]):
                M[j] = M[j] + 2*(B/A)*b[j]
                D[j,j] = D[j,j] + 2*(B/A)

        bd = solve(D,M).flatten() #need to flatten to convert to 1D array
        nm = LA.norm(bd0-bd)
    
    b_subtract = b-bd
    return b_subtract,bd

#### functions for finding peaks and calculating areas

In [ ]:
from scipy.signal import find_peaks

###returns indices for peaks for a given trace as well as the values at the peaks
def find_trace_peaks(trace, min_distance=100, min_height=2.5):
    
    peak_idx, _ = find_peaks(trace, distance=min_distance, height=min_height)  
    peak_val = trace[peak_idx]

    return peak_idx, peak_val

In [ ]:
def return_peak_areas(start_nt, end_nt, trace, trace_nt, ctrl_start=200, ctrl_end=300):
    #start_nt and end_nt should flank the peak of interest
    #trace refers to the reading values (FU on the Bioanalyzer)
    #trace_nt refers to the x-axis, or the nucleotides corresponding to different values in trace
    #ctrl_start, and ctrl_end refer to P4P6 control, and is flanking a wider nucleotide window to account for accumulation
    #of background, degraded RNAs
    
    #indices for P4P6
    p4p6_start_idx = min(range(len(trace_nt)), key=lambda i: abs(trace_nt[i]-ctrl_start))
    p4p6_end_idx = min(range(len(trace_nt)), key=lambda i: abs(trace_nt[i]-ctrl_end))

    #indices for mRNA
    first_idx = min(range(len(trace_nt)), key=lambda i: abs(trace_nt[i]-start_nt))
    last_idx = min(range(len(trace_nt)), key=lambda i: abs(trace_nt[i]-end_nt)) 
    
    #calculating areas
    p4p6_area = np.trapz(y=trace[p4p6_start_idx:p4p6_end_idx], x=trace_nt[p4p6_start_idx:p4p6_end_idx])
    background_area = np.trapz(y=[trace[first_idx], trace[last_idx]], x=[trace_nt[first_idx],trace_nt[last_idx]])
    total_area = np.trapz(y=trace[first_idx:last_idx], x=trace_nt[first_idx:last_idx])
    
    subtracted_area = total_area-background_area
    normalized_area = subtracted_area/p4p6_area
    
    return p4p6_area, background_area, total_area, subtracted_area, normalized_area

def return_total_area(start_nt, end_nt, trace, trace_nt, ctrl_start=200, ctrl_end = 300, total_start = 200):
    '''
    start_nt and end_nt should flank the peak of interest
    trace: signal (FU on the Bioanalyzer)
    trace_nt: nucleotides corresponding to trace
    ctrl_start: nucleotide to start measuring background from for entire trace
    '''
    #indices for P4P6
    p4p6_start_idx = min(range(len(trace_nt)), key=lambda i: abs(trace_nt[i]-ctrl_start))
    p4p6_end_idx = min(range(len(trace_nt)), key=lambda i: abs(trace_nt[i]-ctrl_end))

    #indices for mRNA
    first_idx = min(range(len(trace_nt)), key=lambda i: abs(trace_nt[i]-start_nt))
    last_idx = min(range(len(trace_nt)), key=lambda i: abs(trace_nt[i]-end_nt)) 
    
    #calculating areas:
    #area for p4p6 control (also sometimes 25 nt area)
    p4p6_area = np.trapz(y=trace[p4p6_start_idx:p4p6_end_idx], x=trace_nt[p4p6_start_idx:p4p6_end_idx])
    #area for background
    peak_background_area = np.trapz(y=[trace[first_idx], trace[last_idx]], x=[trace_nt[first_idx],trace_nt[last_idx]])
    #peak area refers to full length peak of interest
    peak_area = np.trapz(y=trace[first_idx:last_idx], x=trace_nt[first_idx:last_idx])
    
    #now calculating full background
    control_start_idx =  min(range(len(trace_nt)), key=lambda i: abs(trace_nt[i]-total_start))
    lane_background_area = np.trapz(y=trace[control_start_idx:last_idx], x=trace_nt[control_start_idx:last_idx])
    
#     #subtract out background and normalize
#     subtracted_area = peak_area-background_area
#     normalized_area = subtracted_area/p4p6_area
    
#     #now also normalize by the total amount of background present
#     subtracted_lane = total_background - subtracted_area #returns the total background for the entire lane
#     normalized_lane = peak_area/total_background #should return ~1 if the majority of the area is represented in peak
    
    return p4p6_area, peak_area, peak_background_area,lane_background_area

def calc_frac_intact(times, norm_areas):
    
    init_val = float(norm_areas[0])
    frac_intact = (norm_areas/init_val).clip(0)
    
    return frac_intact

In [ ]:
from scipy.optimize import curve_fit

#returns indices, bootstrapped
def bootstrap_inds(x):
    bs_indices = np.random.choice(range(len(x)),len(x))
    return bs_indices

#function for exponential fitting
def func(x, A, b, c):
    return A*np.exp(-b*x)+c

def exp_fit(frac_intact, timepoints, func, bs_iter = 1000, p0=(0.8,0.5,0)):
    fit_coeffs = []
    for i in np.arange(0,bs_iter,1):
        
        #just in case
        frac_intact = np.clip(a=frac_intact, a_min=0, a_max=max(frac_intact))
        
        #generate bootstrap indices
        bs_indices = bootstrap_inds(frac_intact)
        
        #generating data for fit
        fit_t = [timepoints[i] for i in bs_indices]
        fit_fracint = [frac_intact[i] for i in bs_indices]
        
        #exponential fit
        popt, pcov = curve_fit(func, fit_t, fit_fracint, maxfev=5000, p0=p0)
        
        fit_coeffs.append((popt, pcov))
        
    return fit_coeffs


def exp_fit_fixed(timepoints, frac_intact, bs_iter = 1000, p0=0.5):
    
    fit_coeffs = []
    
    for i in np.arange(0,bs_iter,1):
        #just in case
        frac_intact = np.clip(a=frac_intact, a_min=0, a_max=max(frac_intact))
        
        #generate bootstrap indices
        bs_indices = bootstrap_inds(frac_intact)
        
        #generating data for fit
        fit_t = [timepoints[i] for i in bs_indices]
        fit_fracint = [frac_intact[i] for i in bs_indices]
        
        #exponential fit, fix exponential decay function to start at value at time point 0
#         popt, pcov = curve_fit(lambda t,b: fit_fracint[0]*np.exp(-1*b*t),  fit_t,  fit_fracint, p0=p0, maxfev=5000, bounds=(0, 1))
        popt, pcov = curve_fit(lambda t,b: np.exp(-1*b*t),  fit_t,  fit_fracint, p0=p0, maxfev=5000)

        fit_coeffs.append((popt, pcov))
    return fit_coeffs

def log_transform_fit(timepoints, frac_intact, bs_iter=1000):
    
    fit_coeffs = []
    
    for i in np.arange(0,bs_iter, 1):
        frac_intact = np.absolute(frac_intact)
        
        #generate bootstrap indices
        bs_indices = bootstrap_inds(frac_intact)
        
        #generating data for fit
        fit_t = [timepoints[i] for i in bs_indices][:8]
        fit_fracint = [frac_intact[i] for i in bs_indices][:8]
        
        #doing a first order fit after log transform
        fit = np.polyfit(fit_t, -1*np.log(fit_fracint), 1, w=np.sqrt(fit_fracint))
        
        fit_coeffs.append(fit[0])
    
    return fit_coeffs


# Analysis of samples starts here:

### appending file names from each sample

In [ ]:
### read in sample map
map_df = pd.read_csv('sample_nucleotide_filename.csv')
# map_df = pd.read_csv('sample_nucleotide_filename_first6.csv')
# map_df

#match plate number to filename:
filenames_df = pd.read_csv('platenumber_filename.csv')
filenames_dict = dict(zip(filenames_df['Plate_Number'],filenames_df['File_Name']))

data_dir = './processed_data/'
#mapping plate number to filename, adding column to map
filenames = []

for filename, filenum in zip(map_df['Plate'], map_df['FileNumber']):
    name = filenames_dict[filename]
    name = 'nts-'+name+'_Sample'+str(filenum)+'.csv'
#     print(name)
    filenames.append(name)

"""
check that files exist
commented out for now after checking, we're good
"""

# for filename in filenames:
#     print(path.exists(data_dir+filename))

map_df['FileName'] = filenames
# map_df

In [ ]:
map_df

In [ ]:
peak_nts_list = []
start_nt_list = []
end_nt_list = []
p4p6_area_list = []
bg_area_list = []
total_area_list = []
subtract_area_list = []
normalized_area_list = []
signal_normalized_area_list = []

plot_dir = './plots/'

peaks_nt_dict = {}
# Iterate through the list of samples, and return a df that has nucleotides and background subtracted values
for row in map_df.itertuples():
    clf()
    #read in dataframe for given sample
    sample_df = pd.read_csv(data_dir+row.FileName)
    
    #extract time series and nucleotides, let's clip to just the first third (up to ~1400 nucleotides)
    array_len = len(sample_df['Nucleotides'])
    clip_len = int(array_len/2.2)
    
    nts = np.array(sample_df['Nucleotides'][:clip_len])
    trace = np.array(sample_df['Value'][:clip_len])
    
    plot(nts, trace, label=row.Sample+'_'+row.Nucleotide+'_'+str(row.Timepoint))
    title(row.Sample+'_'+row.Nucleotide+'_'+str(row.Timepoint))
    xlabel('Nucleotides')
    ylabel('Signal (FU)')
    tight_layout()
    
    savefig(plot_dir+row.Sample+'_'+row.Nucleotide+'_'+str(row.Timepoint)+'.png', dpi=300)
    clf()
    
    ###plotting the background subtracted trace
    trace_norm,_ = baseline_xi(trace)
    plot(nts, trace_norm, label=row.Sample+'_'+row.Nucleotide+'_'+str(row.Timepoint))
    title(row.Sample+'_'+row.Nucleotide+'_'+str(row.Timepoint))
    xlabel('Nucleotides')
    ylabel('Signal (FU)')
    tight_layout()
    
#     savefig(plot_dir+'normalized-'+row.Sample+'_'+row.Nucleotide+'_'+str(row.Timepoint)+'.png', dpi=300)
    clf()
    
    
    if (row.Timepoint == 0):
        peak_idx, peak_val = find_trace_peaks(trace,min_distance=100, min_height=1)
        peak_nts = nts[peak_idx]
        peak_nts_list.append(peak_nts)
        
        start_nt = nts[peak_idx][-1]-100
        end_nt = nts[peak_idx][-1]+100
        
        start_nt_list.append(start_nt)
        end_nt_list.append(end_nt)
        
        peak_assign_dict = {}
        peak_assign_dict['start_nt'] = start_nt
        peak_assign_dict['end_nt'] = end_nt
        peak_assign_dict['peaks'] = peak_nts
        
        peaks_nt_dict[(row.Sample, row.Nucleotide)] = peak_assign_dict

    else:
        time_0_dict = peaks_nt_dict[(row.Sample, row.Nucleotide)]
        peak_nts_list.append(time_0_dict['peaks'])
        start_nt_list.append(time_0_dict['start_nt'])
        end_nt_list.append(time_0_dict['end_nt'])
        
        start_nt = time_0_dict['start_nt']
        end_nt = time_0_dict['end_nt']


#     #integrate at specified nucleotides per sample
#     start_nt = nts[peak_idx][-1]-100
#     end_nt = nts[peak_idx][-1]+100
#     start_nt_list.append(start_nt)
#     end_nt_list.append(end_nt)
    
    p4p6, background, total, subtract, normalized = return_peak_areas(start_nt, end_nt, trace, nts, ctrl_start=20, ctrl_end=30)
    p4p6_area_list.append(p4p6)
    bg_area_list.append(background)
    total_area_list.append(total)
    subtract_area_list.append(subtract)
    normalized_area_list.append(normalized)
    
    _,_,control_area_25, _, _ = return_peak_areas(start_nt=5, end_nt = 50, trace=trace, trace_nt=nts)
    double_normalized = normalized/control_area_25
    signal_normalized_area_list.append(double_normalized)

map_df = map_df.assign(peak_nts = peak_nts_list, start_nt = start_nt_list, end_nt = end_nt_list,\
              p4p6_area = p4p6_area_list, background_area = bg_area_list, total_area = total_area_list,\
              subtracted_area = subtract_area_list, normalized_area = normalized_area_list, double_normalized = signal_normalized_area_list)

map_df
#export dataframe to .csv for recordkeeping
map_df.to_csv('12-10-2020_analyzed_samples_doublenormalized.csv')
# map_df.to_csv('12-10-2020_analyzed_samples_doublenormalized_first6.csv')

#plot configuration
# title('Background Subtracted Traces')
# xlabel('Nucleotides')
# ylabel('Signal (FU)')
# tight_layout()
# savefig('10-21-2020_traces.png', dpi=300)

In [ ]:
#all combinations of sample and nucleotide type
samples = set(zip(map_df['Sample'], map_df['Nucleotide']))

sample_dfs = []
sample_fits = {}
all_fits = {}
for sample in sorted(samples):
    rna_sample = sample[0]
    nucleotide = sample[1]

    working_df = map_df[(map_df['Sample']==rna_sample) & (map_df['Nucleotide']==nucleotide)]
#     working_df
    
    norm_areas = np.array(working_df['normalized_area'])
#     norm_areas = np.array(working_df['double_normalized'])
    times = np.array(working_df['Timepoint'])
    
    frac_intact = calc_frac_intact(times, norm_areas)
    working_df['Frac_Intact'] = frac_intact
    
    plot(times, frac_intact, label=sample, linewidth=3)
    
    fit_dict = {}
    
    try:
        print('Trying an exponential fit...'+str(sample))
        fits = exp_fit(timepoints=times, frac_intact=frac_intact, func=func)
        kdeg = np.mean(fits)
        kdeg_err = np.std(fits)
        print(kdeg)
        print(kdeg_err)
        fit_dict['kdeg'] = kdeg
        fit_dict['kdeg_err'] = kdeg_err
        
    except RuntimeError:
        print('Could not converge for...'+str(sample))
        fit_dict['kdeg'] = 'Error'
        fit_dict['kdeg_err'] = 'Error'
        continue
    
    sample_fits[sample] = fit_dict
    all_fits[sample] = fits
# sample_fits
legend(loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=14)
title('Fraction Intact')
xlabel('Time (hours)')
ylabel('Fraction Intact')

tight_layout()

# savefig

In [ ]:
fit_df = pd.DataFrame.from_dict(sample_fits, orient='index')
fit_df.to_csv('12-10-2020_exponential_fits.csv')

### normaling each lane by how much degradation product exists aka % intact per lane

### TO DO:
- normalize per lane, based on how much degradation product in each lane
- for each lane, normalize intensity by what % of product is desired band
- then divide` % full length product by the % at 0 hrs (should be ~1) to get fraction intact over time
- at the end, print scatterplot of fraction intact over time, and then the exponential fit (average from bootstrap method)


In [ ]:
from tqdm import tqdm

In [ ]:
#let's take the first sample from lane_df
# test_df = map_df[map_df['Sample']=='hHBB_10422827_Ribotree_Random_sup_1_hHBB']
# test_df

peak_nts_list = []
start_nt_list = []
end_nt_list = []
p4p6_area_list = []
peak_area_list = []
peak_background_area_list = []
lane_background_area_list = []
peaks_nt_dict = {}

#iterating through the dataframe
for row in map_df.itertuples():
    
    sample_df = pd.read_csv(data_dir+row.FileName)
    
    #extract time series and nucleotides, let's clip to just the first third (up to ~1400 nucleotides)
    array_len = len(sample_df['Nucleotides'])
    clip_len = int(array_len/2.2)
    
    nts = np.array(sample_df['Nucleotides'][:clip_len])
    trace = np.array(sample_df['Value'][:clip_len])
    
    ###plotting the background subtracted trace
    trace_norm,_ = baseline_xi(trace)
    
    if (row.Timepoint == 0):
        peak_idx, peak_val = find_trace_peaks(trace,min_distance=100, min_height=1)
        peak_nts = nts[peak_idx]
        peak_nts_list.append(peak_nts)
        
        start_nt = nts[peak_idx][-1]-100
        end_nt = nts[peak_idx][-1]+100
        
        start_nt_list.append(start_nt)
        end_nt_list.append(end_nt)
        
        peak_assign_dict = {}
        peak_assign_dict['start_nt'] = start_nt
        peak_assign_dict['end_nt'] = end_nt
        peak_assign_dict['peaks'] = peak_nts
        
        peaks_nt_dict[(row.Sample, row.Nucleotide)] = peak_assign_dict

    else:
        time_0_dict = peaks_nt_dict[(row.Sample, row.Nucleotide)]
        peak_nts_list.append(time_0_dict['peaks'])
        start_nt_list.append(time_0_dict['start_nt'])
        end_nt_list.append(time_0_dict['end_nt'])
        
        start_nt = time_0_dict['start_nt']
        end_nt = time_0_dict['end_nt']
    
    p4p6_area, peak_area, peak_background_area, lane_background_area = return_total_area(start_nt, end_nt, trace, nts, ctrl_start=15, ctrl_end=40)

    
    p4p6_area_list.append(p4p6_area)
    peak_area_list.append(peak_area)
    peak_background_area_list.append(peak_background_area)
    lane_background_area_list.append(lane_background_area)
    
#     _,_,control_area_25, _, _ = return_peak_areas(start_nt=5, end_nt = 50, trace=trace, trace_nt=nts)
#     double_normalized = normalized/control_area_25
#     signal_normalized_area_list.append(double_normalized)
    
    
map_df = map_df.assign(peak_nts = peak_nts_list, 
                         start_nt = start_nt_list, 
                         end_nt = end_nt_list,
                         p4p6_area = p4p6_area_list,
                         peak_area = peak_area_list,
                         peak_background_area = peak_background_area_list,
                         lane_background_area = lane_background_area_list)
map_df = map_df.dropna(axis=1)
map_df.to_csv('map_allareas.csv')

In [ ]:
map_df['peak_area_subtracted'] = map_df['peak_area']-map_df['peak_background_area']
map_df['fraction_intact'] = map_df['peak_area_subtracted']/map_df['lane_background_area']

In [ ]:
matplotlib.rcParams['pdf.fonttype'] = 42
plot_dir = './plots/'

#all combinations of sample and nucleotide type
samples = set(zip(map_df['Sample'], map_df['Nucleotide']))
# samples=samples[0]

sample_dfs = []
sample_fits = {}
all_fits = {}

num_plots = len(samples)
num_rows = round(num_plots/4, 1)
num_columns = num_plots/num_rows
figure(figsize=(num_rows*3+5, num_columns*4+2))
sample_dfs = []
sample_fits = {}
all_fits = {}

for i, sample in tqdm(enumerate(sorted(samples))):
    
    subplot(num_rows, num_columns, i+1)
    
    rna_sample = sample[0]
    nucleotide = sample[1]

    #extracting the df for that sample, nucleotide combo
    working_df = map_df[(map_df['Sample']==rna_sample) & (map_df['Nucleotide']==nucleotide)]
    
    #times are time points (t), fraction intact values (fi)
    #to be used in fi = np.exp(-b*t) fit for b coefficient --> kdeg calculation
    times = np.array(working_df['Timepoint'])[:8]
    frac_intact = np.array(working_df['fraction_intact'])[:8]

    scatter(times, frac_intact, label='Data', s=20, marker='o')
    
    fit_dict = {}
    try:
        print('Trying an exponential fit...'+str(sample))
        fits = np.array(log_transform_fit(timepoints = times, frac_intact=frac_intact, bs_iter=1000))
        kdeg = np.average(fits)
        kdeg_err = np.std(fits)
        print('kdeg: '+str(kdeg))
        print('kdeg_err: '+str(kdeg_err))
        fit_dict['kdeg'] = kdeg
        fit_dict['kdeg_err'] = kdeg_err
        
#         plotting fit
        plot(np.arange(0,24,0.05), frac_intact[0]*np.exp(-1*kdeg*np.arange(0,24,0.05)), linewidth=3, label='Fit')
        
    except RuntimeError:
        print('Could not converge for...'+str(sample))
        fit_dict['kdeg'] = 'Error'
        fit_dict['kdeg_err'] = 'Error'
        continue
    
    sample_fits[sample] = fit_dict
    all_fits[sample] = fits
    
    legend(loc='upper left', bbox_to_anchor=(1.05,1), fontsize=14)
    title('{}'.format(rna_sample), fontsize=12)
    xlabel('Time (hours)')
    ylabel('Fraction Intact')
    tight_layout()

savefig(plot_dir+'numpy_exponential_fit.pdf')
#     clf()

In [ ]:
pd.DataFrame.from_dict(sample_fits, orient='index').to_csv('12-10_expfits.csv')

In [ ]:
expfit_1210_df = pd.read_csv('12-10_expfits.csv')

#returning rank
kdeg_1210 = expfit_1210_df['kdeg_12-10']
kdeg_1202 = expfit_1210_df['kdeg_12-02']

def array_rank(array):
    temp = array.argsort()
    ranks = numpy.empty_like(temp)
    ranks[temp] = numpy.arange(len(array))
    return ranks

expfit_1210_df['rank_12-10'] = array_rank(kdeg_1210)
expfit_1210_df['rank_12-02'] = array_rank(kdeg_1202)

# expfit_1210_df

sns.scatterplot(data=expfit_1210_df, x='rank_12-10', y='rank_12-02', s=50)
title('Comparing relative ranks', fontsize=14)
xlabel('Relative Rank (12-10)', fontsize=14)
ylabel('Relative Rank (12-02)', fontsize=14)

In [ ]:
#performing Kendall-Tau:
kt_coeff, kt_pval = sp.stats.kendalltau(expfit_1210_df['rank_12-10'], expfit_1210_df['rank_12-02'])
kt_coeff
kt_pval

In [ ]:
expfit_1210_df.to_csv('kdeg_ranking.csv')

In [ ]:
sns.distplot(expfit_1210_df['kdeg_12-10'], label='12-10 (10 time points)')
sns.distplot(expfit_1210_df['kdeg_12-02'], label='12-02 (4 time points)')
legend()

In [ ]:
test_df=map_df[map_df['Sample']=='hHBB_10383581_START_reference_hHBB']
test_df['fraction_intact'] = test_df['fraction_intact'].clip(0)

sns.scatterplot(data=test_df, x='Timepoint', y='fraction_intact')

times = np.array(test_df['Timepoint'])
frac_intact = np.array(test_df['fraction_intact'])

# times
# frac_intact

fits = exp_fit_fixed(timepoints=times, frac_intact=frac_intact, bs_iter=1000)


# fits

scatter(times, frac_intact,marker='x', s=100)
coeffs = []

for i,fit in enumerate(fits):
    y=np.exp(-1*kdeg*times)
    plot(times, y)
    coeffs.append(fit)
#     kdeg = fit[0]
#     if kdeg<0:
#         print(i)
#         print(fit)
#         pass
#     else:
#         y=frac_intact[0]*np.exp(-1*kdeg*times)
#         plot(times, y)
#         coeffs.append(fit)
        
# legend()

# coeffs

### subplot of log transformed fraction intact values to check linearity

In [ ]:
samples = set(zip(map_df['Sample'], map_df['Nucleotide']))

num_plots = len(samples)
num_rows = round(num_plots/4, 1)
num_columns = num_plots/num_rows
figure(figsize=(num_rows*3+5, num_columns*4+2))
# sample_dfs = []
# sample_fits = {}
# all_fits = {}

for i, sample in tqdm(enumerate(sorted(samples))):
    
    subplot(num_rows, num_columns, i+1)
    rna_sample = sample[0]
    nucleotide = sample[1]

    #extracting the df for that sample, nucleotide combo
    working_df = map_df[(map_df['Sample']==rna_sample) & (map_df['Nucleotide']==nucleotide)]
    
    #times are time points (t), fraction intact values (fi)
    #to be used in fi = np.exp(-b*t) fit for b coefficient --> kdeg calculation
    times = np.array(working_df['Timepoint'])[:8]
    frac_intact = np.array(working_df['fraction_intact'])[:8]
    
#     clf()
    scatter(times, np.log(frac_intact), s=20, marker='o')
    
    title('{}'.format(rna_sample), fontsize=12)
    xlabel('Time (hours)')
    ylabel('ln(Fraction Intact)')

tight_layout()

# savefig(plot_dir+'logy.pdf')

In [ ]:
test_df=map_df[map_df['Sample']=='hHBB_10383581_START_reference_hHBB']
# test_df['fraction_intact'] = test_df['fraction_intact'].clip()

# sns.scatterplot(data=test_df, x='Timepoint', y='fraction_intact')

times = np.array(test_df['Timepoint'])
frac_intact = np.array(test_df['fraction_intact'])

test_df['fraction_intact']
frac_intact
-1*np.log(frac_intact)

scatter(times, -1*np.log(frac_intact))
ylabel('-log(fraction intact)', fontsize=14)
xlabel('Hours')
xticks(fontsize=12)
yticks(fontsize=12)
tight_layout()

# np.polyfit(times, np.log(frac_intact), 1, w=np.sqrt(frac_intact))
# np.polyfit(times, -1*np.log(frac_intact), 1)

# scatter(times, frac_intact,marker='x', s=100)
# coeffs = []

# for i,fit in enumerate(fits):
#     y=np.exp(-1*kdeg*times)
#     plot(times, y)
#     coeffs.append(fit)
#     kdeg = fit[0]
#     if kdeg<0:
#         print(i)
#         print(fit)
#         pass
#     else:
#         y=frac_intact[0]*np.exp(-1*kdeg*times)
#         plot(times, y)
#         coeffs.append(fit)
        
# legend()


In [ ]:
average_coeffs = np.average(coeffs, axis=0)
float(average_coeffs[0])

scatter(times, frac_intact,marker='x', s=100)
plot(times, frac_intact[0]*np.exp(-1*float(average_coeffs[0])*times))
average_coeffs

In [ ]:
coeffs

In [ ]:
init_areas = {}
for sample in samples:
    sample_name = sample[0]
    sample_nt = sample[1]
    init_areas[sample] = float(map_df[(map_df['Timepoint']==0)&(map_df['Sample']==sample_name)
                               &(map_df['Nucleotide']==sample_nt)]['normalized_area'])

frac_intact_list = []
for row in map_df.itertuples():
    sample_nt = (row.Sample, row.Nucleotide)
    frac_intact = np.maximum(row.normalized_area/init_areas[sample_nt], 0)
    frac_intact_list.append(frac_intact)

map_df['Fraction_intact'] = frac_intact_list

map_df.to_csv('12-10-2020_fraction_intact.csv')

In [ ]:
map_df

### flexible peak identification

In [ ]:
flex_map_df = map_df

peak_nts_list = []
all_peak_nts = []
start_nt_list = []
end_nt_list = []
p4p6_area_list = []
peak_area_list = []
peak_background_area_list = []
lane_background_area_list = []
peaks_nt_dict = {}

    
# iterating through the dataframe
for i,row in enumerate(flex_map_df.itertuples()):
    
    sample_df = pd.read_csv(data_dir+row.FileName)
    
    #extract time series and nucleotides, let's clip to just the first third (up to ~1400 nucleotides)
    array_len = len(sample_df['Nucleotides'])
    clip_len = int(array_len/2.2)
    
    nts = np.array(sample_df['Nucleotides'][:clip_len])
    trace = np.array(sample_df['Value'][:clip_len])
    
    ###plotting the background subtracted trace
    trace_norm,_ = baseline_xi(trace)
    
    peak_idx, peak_val = find_trace_peaks(trace,min_distance=100, min_height=1)
    
    all_peak_nts.append(nts[peak_idx])
    
    #checking if a real peak exists:
    if nts[peak_idx][-1]>800:
        peak_nts = nts[peak_idx][-1]
    else:
        peak_nts = peak_nts_list[-1]
        
    peak_nts_list.append(peak_nts)
    
    start_nt = peak_nts-100
    end_nt = peak_nts+100
        
    start_nt_list.append(start_nt)
    end_nt_list.append(end_nt)

    peak_assign_dict = {}
    peak_assign_dict['start_nt'] = start_nt
    peak_assign_dict['end_nt'] = end_nt
    peak_assign_dict['peaks'] = peak_nts

    peaks_nt_dict[(row.Sample, row.Nucleotide)] = peak_assign_dict    
    p4p6_area, peak_area, peak_background_area, lane_background_area = return_total_area(start_nt, end_nt, trace, nts, ctrl_start=15, ctrl_end=40)

    p4p6_area_list.append(p4p6_area)
    peak_area_list.append(peak_area)
    peak_background_area_list.append(peak_background_area)
    lane_background_area_list.append(lane_background_area)
    
flex_map_df = flex_map_df.assign(peak_nts = peak_nts_list, 
                         start_nt = start_nt_list, 
                         end_nt = end_nt_list,
                         p4p6_area = p4p6_area_list,
                         peak_area = peak_area_list,
                         peak_background_area = peak_background_area_list,
                         lane_background_area = lane_background_area_list,
                         all_id_peaks = all_peak_nts)

# flex_map_df

flex_map_df['peak_area_subtracted'] = flex_map_df['peak_area']-flex_map_df['peak_background_area']
flex_map_df['fraction_intact'] = flex_map_df['peak_area_subtracted']/flex_map_df['lane_background_area']
# map_df = map_df.dropna(axis=1)
flex_map_df.to_csv('flex_map_allareas.csv')

In [ ]:
samples = set(zip(flex_map_df['Sample'], flex_map_df['Nucleotide']))

num_plots = len(samples)
num_rows = round(num_plots/4, 1)
num_columns = num_plots/num_rows
figure(figsize=(num_rows*3+5, num_columns*4+2))
# sample_dfs = []
# sample_fits = {}
# all_fits = {}

for i, sample in tqdm(enumerate(sorted(samples))):
    
    subplot(num_rows, num_columns, i+1)
    rna_sample = sample[0]
    nucleotide = sample[1]

    #extracting the df for that sample, nucleotide combo
    working_df = flex_map_df[(flex_map_df['Sample']==rna_sample) & (flex_map_df['Nucleotide']==nucleotide)]
    
    #times are time points (t), fraction intact values (fi)
    #to be used in fi = np.exp(-b*t) fit for b coefficient --> kdeg calculation
    times = np.array(working_df['Timepoint'])[:8]
    frac_intact = np.array(working_df['fraction_intact'])[:8]
    
#     clf()
    scatter(times, np.log(frac_intact), s=20, marker='o')
    
    title('{}'.format(rna_sample), fontsize=12)
    xlabel('Time (hours)')
    ylabel('ln(Fraction Intact)')

tight_layout()

savefig(plot_dir+'flex_peaks_logy.pdf')

In [ ]:
matplotlib.rcParams['pdf.fonttype'] = 42
plot_dir = './plots/'

#all combinations of sample and nucleotide type
samples = set(zip(flex_map_df['Sample'], flex_map_df['Nucleotide']))
# samples=samples[0]

sample_dfs = []
sample_fits = {}
all_fits = {}

num_plots = len(samples)
num_rows = round(num_plots/4, 1)
num_columns = num_plots/num_rows
figure(figsize=(num_rows*3+5, num_columns*4+2))
sample_dfs = []
sample_fits = {}
all_fits = {}

for i, sample in tqdm(enumerate(sorted(samples))):
    
    subplot(num_rows, num_columns, i+1)
    
    rna_sample = sample[0]
    nucleotide = sample[1]

    #extracting the df for that sample, nucleotide combo
    working_df = flex_map_df[(flex_map_df['Sample']==rna_sample) & (flex_map_df['Nucleotide']==nucleotide)]
    
    #times are time points (t), fraction intact values (fi)
    #to be used in fi = np.exp(-b*t) fit for b coefficient --> kdeg calculation
    times = np.array(working_df['Timepoint'])[:8]
    frac_intact = np.array(working_df['fraction_intact'])[:8]

    scatter(times, frac_intact, label='Data', s=20, marker='o')
    
    fit_dict = {}
    try:
        print('Trying an exponential fit...'+str(sample))
        fits = np.array(log_transform_fit(timepoints = times, frac_intact=frac_intact, bs_iter=1000))
        kdeg = np.average(fits)
        kdeg_err = np.std(fits)
        print('kdeg: '+str(kdeg))
        print('kdeg_err: '+str(kdeg_err))
        fit_dict['kdeg'] = kdeg
        fit_dict['kdeg_err'] = kdeg_err
        
#         plotting fit
        plot(np.arange(0,24,0.05), frac_intact[0]*np.exp(-1*kdeg*np.arange(0,24,0.05)), linewidth=3, label='Fit')
        
    except RuntimeError:
        print('Could not converge for...'+str(sample))
        fit_dict['kdeg'] = 'Error'
        fit_dict['kdeg_err'] = 'Error'
        continue
    
    sample_fits[sample] = fit_dict
    all_fits[sample] = fits
    
    legend(loc='upper left', bbox_to_anchor=(1.05,1), fontsize=14)
    title('{}'.format(rna_sample), fontsize=12)
    xlabel('Time (hours)')
    ylabel('Fraction Intact')
    tight_layout()

savefig(plot_dir+'flex_peaks_numpy_exponential_fit.pdf')
#     clf()

In [ ]:
pd.DataFrame.from_dict(sample_fits, orient='index').to_csv('12-10_flex_peaks_expfits.csv')

### only using four time points like 12-02 
#### 0, 1, 3, 5 hours

In [ ]:
### read in sample map
map_4tps_df = pd.read_csv('sample_nucleotide_filename.csv')
map_4tps_df = map_4tps_df[map_4tps_df['Timepoint'].isin([0.0, 1.0, 3.0, 5.0])]

#match plate number to filename:
filenames_df = pd.read_csv('platenumber_filename.csv')
filenames_dict = dict(zip(filenames_df['Plate_Number'],filenames_df['File_Name']))

data_dir = './processed_data/'
#mapping plate number to filename, adding column to map
filenames = []

for filename, filenum in zip(map_4tps_df['Plate'], map_4tps_df['FileNumber']):
    name = filenames_dict[filename]
    name = 'nts-'+name+'_Sample'+str(filenum)+'.csv'
    filenames.append(name)

map_4tps_df['FileName'] = filenames

peak_nts_list = []
all_peak_nts = []
start_nt_list = []
end_nt_list = []
p4p6_area_list = []
peak_area_list = []
peak_background_area_list = []
lane_background_area_list = []
peaks_nt_dict = {}
    
# iterating through the dataframe
for i,row in enumerate(map_4tps_df.itertuples()):
    
    sample_df = pd.read_csv(data_dir+row.FileName)
    
    #extract time series and nucleotides, let's clip to just the first third (up to ~1400 nucleotides)
    array_len = len(sample_df['Nucleotides'])
    clip_len = int(array_len/2.2)
    
    nts = np.array(sample_df['Nucleotides'][:clip_len])
    trace = np.array(sample_df['Value'][:clip_len])
    
    ###plotting the background subtracted trace
    trace_norm,_ = baseline_xi(trace)
    
    peak_idx, peak_val = find_trace_peaks(trace,min_distance=100, min_height=1)
    
    all_peak_nts.append(nts[peak_idx])
    
    #checking if a real peak exists:
    if nts[peak_idx][-1]>800:
        peak_nts = nts[peak_idx][-1]
    else:
        peak_nts = peak_nts_list[-1]
        
    peak_nts_list.append(peak_nts)
    
    start_nt = peak_nts-100
    end_nt = peak_nts+100
        
    start_nt_list.append(start_nt)
    end_nt_list.append(end_nt)

    p4p6_area, peak_area, peak_background_area, lane_background_area = return_total_area(start_nt, end_nt, trace, nts, ctrl_start=15, ctrl_end=40)

    p4p6_area_list.append(p4p6_area)
    peak_area_list.append(peak_area)
    peak_background_area_list.append(peak_background_area)
    lane_background_area_list.append(lane_background_area)
    
map_4tps_df = map_4tps_df.assign(peak_nts = peak_nts_list, 
                         start_nt = start_nt_list, 
                         end_nt = end_nt_list,
                         p4p6_area = p4p6_area_list,
                         peak_area = peak_area_list,
                         peak_background_area = peak_background_area_list,
                         lane_background_area = lane_background_area_list,
                         all_id_peaks = all_peak_nts)

map_4tps_df['peak_area_subtracted'] = map_4tps_df['peak_area']-map_4tps_df['peak_background_area']
map_4tps_df['fraction_intact'] = map_4tps_df['peak_area_subtracted']/map_4tps_df['lane_background_area']
map_4tps_df = map_4tps_df.dropna(axis=1)
map_4tps_df.to_csv('map_4tps_map_allareas.csv')

matplotlib.rcParams['pdf.fonttype'] = 42
plot_dir = './plots/'

#all combinations of sample and nucleotide type
samples = set(zip(flex_map_df['Sample'], flex_map_df['Nucleotide']))
# samples=samples[0]

sample_dfs = []
sample_fits = {}
all_fits = {}

num_plots = len(samples)
num_rows = round(num_plots/4, 1)
num_columns = num_plots/num_rows
figure(figsize=(num_rows*3+5, num_columns*4+2))
sample_dfs = []
sample_fits = {}
all_fits = {}

for i, sample in tqdm(enumerate(sorted(samples))):
    
    subplot(num_rows, num_columns, i+1)
    
    rna_sample = sample[0]
    nucleotide = sample[1]

    #extracting the df for that sample, nucleotide combo
    working_df = flex_map_df[(flex_map_df['Sample']==rna_sample) & (flex_map_df['Nucleotide']==nucleotide)]
    
    #times are time points (t), fraction intact values (fi)
    #to be used in fi = np.exp(-b*t) fit for b coefficient --> kdeg calculation
    times = np.array(working_df['Timepoint'])[:8]
    frac_intact = np.array(working_df['fraction_intact'])[:8]

    scatter(times, frac_intact, label='Data', s=20, marker='o')
    
    fit_dict = {}
    try:
        print('Trying an exponential fit...'+str(sample))
        fits = np.array(log_transform_fit(timepoints = times, frac_intact=frac_intact, bs_iter=1000))
        kdeg = np.average(fits)
        kdeg_err = np.std(fits)
        print('kdeg: '+str(kdeg))
        print('kdeg_err: '+str(kdeg_err))
        fit_dict['kdeg'] = kdeg
        fit_dict['kdeg_err'] = kdeg_err
        
#         plotting fit
        plot(np.arange(0,24,0.05), frac_intact[0]*np.exp(-1*kdeg*np.arange(0,24,0.05)), linewidth=3, label='Fit')
        
    except RuntimeError:
        print('Could not converge for...'+str(sample))
        fit_dict['kdeg'] = 'Error'
        fit_dict['kdeg_err'] = 'Error'
        continue
    
    sample_fits[sample] = fit_dict
    all_fits[sample] = fits
    
    legend(loc='upper left', bbox_to_anchor=(1.05,1), fontsize=14)
    title('{}'.format(rna_sample), fontsize=12)
    xlabel('Time (hours)')
    ylabel('Fraction Intact')
    tight_layout()

savefig(plot_dir+'flex_peaks_4tps_numpy_exponential_fit.pdf')
#     clf()

pd.DataFrame.from_dict(sample_fits, orient='index').to_csv('12-10_flex_peaks_4tps_expfits.csv')

In [ ]:
all_fits_df = pd.DataFrame.from_dict(all_fits, orient='index')
all_fits_df = all_fits_df.reset_index().rename({'index':'Sample'})
all_fits_df_long = pd.melt(all_fits_df, id_vars=['index'], value_vars=np.arange(1000))
all_fits_df_long['Sample'] = [str(sample[0]) for sample in all_fits_df_long['index']]
all_fits_df_long['Nucleotide'] = [str(sample[1]) for sample in all_fits_df_long['index']]
all_fits_df_long['kdeg'] = [float(x) for x in all_fits_df_long['value']]

kdeg_df_long = all_fits_df_long[['Sample', 'Nucleotide', 'kdeg']]


figure(figsize=(10,6))
sns.barplot(data=kdeg_df_long, x='Sample', y='kdeg', hue='Nucleotide', ci='sd', palette='cividis')
xlabel('RNA')
ylabel('kdeg (hr-1)')
legend(title='Nucleotide Type',loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=14)
xticks(ha='right', rotation=45)
tight_layout()
savefig(plot_dir+'kdeg_fits.png', dpi=300)


In [ ]:
# kdeg_df_long

seqid_names = pd.read_csv('seqid_names.csv')
seqid_names_dict = dict(zip(seqid_names['Human readable name'], seqid_names['Barcode']))
# seqid_names_dict

seqid_list = []
for row in kdeg_df_long.itertuples():
    seqid_list.append(seqid_names_dict[row.Sample])
    
kdeg_df_long['seqid'] = seqid_list

# kdeg_df_long

figure(figsize=(10,6))
sns.barplot(data=kdeg_df_long.sort_values(by='seqid', ascending=True), x='Sample', y='kdeg', hue='Nucleotide', ci='sd', palette='cividis')
xlabel('RNA')
ylabel('kdeg (hr-1)')
ylim(0,1)
legend(title='Nucleotide Type',loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=14)
xticks(ha='right', rotation=45)
tight_layout()
savefig(plot_dir+'kdeg_fits_ordered_seqid.png', dpi=300)


In [ ]:
# fits_df = pd.DataFrame(sample_fits).T
# fits_df = fits_df.reset_index()

# fits_df.rename({'level_0': 'Sample', 'level_1': 'Nucleotide'})

# sns.barplot(data=fits_df, y='kdeg', x='level_0', hue='level_1')